# Mini Project 3: Social Networking Platform

Neo4j Python Documentation: 
- https://neo4j.com/docs/api/python-driver/current/api.html
- https://neo4j.com/docs/cypher-manual/current/clauses/unwind/

## Import Libraries

In [82]:
# !pip install neo4j
import neo4j
import pandas as pd
from neo4j import GraphDatabase

## Define Neo4j Connection & Clear Functions

In [83]:
def connect_db():
    driver = neo4j.GraphDatabase.driver(uri="neo4j://0.0.0.0:7687", auth=("neo4j","password"))
    session = driver.session(database="neo4j")
    return session

# Function to execute query and return results in a pandas DataFrame
def run_query(query):
    driver = neo4j.GraphDatabase.driver(uri="neo4j://0.0.0.0:7687", auth=("neo4j","password"))
    with driver.session() as session:
        results = session.run(query)
        return pd.DataFrame([record.values() for record in results], columns=results.keys())
    
def wipe_out_db(session):
    # wipe out database by deleting all nodes and relationships
    
    # similar to SELECT * FROM graph_db in SQL
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [84]:
session = connect_db()
wipe_out_db(session) 

## Notes

#### CREATE
- Creating a node:
    - Ex.: `CREATE (nodeVar:NodeLabel {prop1: 'prop1 value', prop2: 'prop2 value'})`
    - ( ) indicates the node
    - `nodeVar:NodeLabel` – nodeVar is the node variable, NodeLabel is the node label
    - { } contains node properties that describe the node
- Creating a relationship:
    - Ex.: `CREATE (nodeVar1)-[:name_of_relationship {propA: 'propA value'}]->(nodeVar2)`


#### MATCH 
- MATCH specifies a pattern of nodes and relationships to query/find
- Syntax: 
```
    MATCH [graph_pattern]
    WHERE [query]
    RETURN [result]
```
- Ex.: 
```
    MATCH (nodeVar:NodeLabel {prop1: 'prop1 value', prop2: 'prop2 value'})
    WHERE nodeVar.prop1 = 'some value'
    RETURN nodeVar
    ORDER BY nodeVar.prop1, nodeVar.prop2
```
- Ex.: 
```
    MATCH (src:NodeLabel)-[relationVar:RelationType]->(dst:NodeLabel)
    WHERE src.prop1 = 'some value'
    RETURN relationVar
```

**Notes**

- MATCH assigns matches to the variable nodeVar.
- WHERE filters the query.
- nodeVar.prop1 = 'some value' compares _prop1_ property to the value _some value_.
- RETURN returns particular results.
- Some built-in Cypher functions to retrieve properties:
    - type(relationVar): retrieves the edge/connection type
    - labels(nodeVar): retrieves the labels of a node


## Data Base Design & Setup

**Nodes:**

1. User: individuals on the platform
- Properties: user_id (unique identifier), name (full name), username (unique username), graduation_year (year of graduation), email (email address), linkedin_profile (LinkedIn profile URL), interest (career interest: data science, data analytics, etc.), company_name (name of current employer), current_position (job title)

2. Post: content shared by users
- Properties: post_id (unique identifier), title (title of the post), body (content of the post), date (date of post creation), type (type of post: text, link, image, video, etc.)

3. Event: events that users can attend
- Properties: event_id (unique identifier), name (name of the event), description (details about the event), date (date of the event), location (venue of the event)

4. Comment: user comments on posts
- Properties: comment_id (unique identifier), text (content of the comment), date (date the comment was posted), seen_status (indicates if the comment has been read)

**Edges:**

1. FRIENDS_WITH (User ↔ User): Indicates a friendship relationship between two users
- Properties: since (date the friendship started)

2. CREATED (User → Post): Indicates which user created a specific post
- No additional properties

3. ATTENDING (User → Event): Shows that a user is going to or interested in an event
- Properties: rsvp_date (date the user confirmed attendance)

4. COMMENTED_ON (User → Comment → Post): Connects a user and their comment to a post
- Properties: date (date the comment was made)

5. FOLLOWING (User → User): Represents one user following another without mutual friendship
- Properties: since (date when the following started)

6. HOSTS (User → Event): Indicates which user is hosting a particular event
- Properties: date (date when the user announced hosting the event)


## Data Insertion

In [85]:
schema_dsi = '''
USER: 

CREATE (:User {
  user_id: "<user_id>", 
  name: "<name>", 
  username: "<username>", 
  graduation_year: <graduation_year>, 
  email: "<email>", 
  linkedin_profile: "<linkedin_profile>", 
  interest: "<interest>", 
  company_name: "<company_name>", 
  current_position: "<current_position>"
})
------------------------
POST: 

CREATE (:Post {
  post_id: "<post_id>", 
  title: "<title>", 
  body: "<body>", 
  date: "<date>", 
  type: "<type>"
})
------------------------
Event: 

CREATE (:Event {
  event_id: "<event_id>", 
  name: "<name>", 
  description: "<description>", 
  date: "<date>", 
  location: "<location>"
})
------------------------
Comment:

CREATE (:Comment {
  comment_id: "<comment_id>", 
  text: "<text>", 
  date: "<date>", 
  seen_status: <seen_status>  // true or false
})
------------------------
------------------------
------------------------
FRIENDS_WITH: 

MATCH (u1:User {user_id: "<user_id1>"}), (u2:User {user_id: "<user_id2>"})
CREATE (u1)-[:FRIENDS_WITH {since: "<since_date>"}]->(u2)
------------------------
CREATED:

MATCH (u:User {user_id: "<user_id>"}), (p:Post {post_id: "<post_id>"})
CREATE (u)-[:CREATED]->(p)
------------------------
ATTENDING: 

MATCH (u:User {user_id: "<user_id>"}), (e:Event {event_id: "<event_id>"})
CREATE (u)-[:ATTENDING {rsvp_date: "<rsvp_date>"}]->(e)
------------------------
COMMENTED_ON:

MATCH (u:User {user_id: "<user_id>"}), (c:Comment {comment_id: "<comment_id>"}), (p:Post {post_id: "<post_id>"})
CREATE (u)-[:COMMENTED_ON {date: "<comment_date>"}]->(c)
CREATE (c)-[:COMMENTED_ON {date: "<comment_date>"}]->(p)
------------------------
FOLLOWING: 

MATCH (u1:User {user_id: "<follower_user_id>"}), (u2:User {user_id: "<followed_user_id>"})
CREATE (u1)-[:FOLLOWING {since: "<since_date>"}]->(u2)
------------------------
HOSTS:

MATCH (u:User {user_id: "<user_id>"}), (e:Event {event_id: "<event_id>"})
CREATE (u)-[:HOSTS {date: "<host_date>"}]->(e)
------------------------
'''

### Nodes

In [86]:
# User nodes
query = '''
CREATE (u1:User {user_id: "U001", name: "Alice Johnson", username: "alicej", graduation_year: 2022, email: "alicej@example.com", linkedin_profile: "https://linkedin.com/in/alicej", interest: "Data Science", company_name: "DataTech Inc.", current_position: "Data Scientist"})
CREATE (u2:User {user_id: "U002", name: "Bob Smith", username: "bobsmith", graduation_year: 2021, email: "bobsmith@example.com", linkedin_profile: "https://linkedin.com/in/bobsmith", interest: "Data Analytics", company_name: "AnalytiX", current_position: "Data Analyst"})
CREATE (u3:User {user_id: "U003", name: "Carol Lee", username: "carollee", graduation_year: 2024, email: "carollee@example.com", linkedin_profile: "https://linkedin.com/in/carollee", interest: "Machine Learning Engineer", company_name: "DSI", current_position: "Student"})
CREATE (u4:User {user_id: "U004", name: "David Green", username: "davidg", graduation_year: 2019, email: "davidg@example.com", linkedin_profile: "https://linkedin.com/in/davidg", interest: "Data Engineering", company_name: "BigData Solutions", current_position: "Big Data Engineer"})
CREATE (u5:User {user_id: "U005", name: "Eva Turner", username: "evat", graduation_year: 2025, email: "evat@example.com", linkedin_profile: "https://linkedin.com/in/evat", interest: "Data Science", company_name: "DSI", current_position: "Student"})
CREATE (u6:User {user_id: "U006", name: "Frank Harris", username: "frankh", graduation_year: 2024, email: "frankh@example.com", linkedin_profile: "https://linkedin.com/in/frankh", interest: "Data Engineering", company_name: "DSI", current_position: "Student"})

'''
session.run(query)

# Post nodes
query = '''
CREATE (p1:Post {post_id: "P001", title: "Introduction to Data Science", body: "Exploring the basics of Data Science...", date: "2023-11-01", type: "text"})
CREATE (p2:Post {post_id: "P002", title: "Latest Trends in Data Analytics", body: "An overview of recent developments in Data Analytics...", date: "2023-11-05", type: "link"})
CREATE (p3:Post {post_id: "P003", title: "Big Data and Its Impact", body: "Discussing the impact of Big Data on various industries...", date: "2023-11-10", type: "video"})
CREATE (p4:Post {post_id: "P004", title: "Visualizing Data Effectively", body: "Tips on creating effective data visualizations...", date: "2023-11-12", type: "image"})
CREATE (p5:Post {post_id: "P005", title: "Understanding Machine Learning Algorithms", body: "An introductory guide to machine learning algorithms and their applications.", date: "2023-11-15", type: "text"})
CREATE (p6:Post {post_id: "P006", title: "The Future of Artificial Intelligence", body: "Exploring the potential future developments and impacts of AI.", date: "2023-11-17", type: "link"})
CREATE (p7:Post {post_id: "P007", title: "Data Privacy in the Digital Age", body: "A comprehensive look at data privacy concerns and solutions.", date: "2023-11-20", type: "text"})
CREATE (p8:Post {post_id: "P008", title: "Top 10 Data Visualization Tools", body: "Ranking and reviewing the top 10 tools for data visualization in 2023.", date: "2023-11-22", type: "link"})
CREATE (p9:Post {post_id: "P009", title: "Blockchain and Data Security", body: "How blockchain technology is revolutionizing data security.", date: "2023-11-25", type: "text"})
CREATE (p10:Post {post_id: "P010", title: "Advancements in Predictive Analytics", body: "Discussing recent advancements and trends in predictive analytics.", date: "2023-11-28", type: "link"})
'''
session.run(query)

# Event nodes
query = '''
CREATE (e1:Event {event_id: "E001", name: "Data Science Meetup 2023", description: "An event for data science enthusiasts to network and learn.", date: "2024-01-15", location: "Tech Park, City Center"})
CREATE (e2:Event {event_id: "E002", name: "AI Conference", description: "Annual conference on Artificial Intelligence and Machine Learning.", date: "2024-02-20", location: "Convention Hall, Downtown"})
CREATE (e3:Event {event_id: "E003", name: "Big Data Conference 2024", description: "A conference focusing on trends and challenges in Big Data.", date: "2024-03-10", location: "Tech Arena, Bay Area"})
CREATE (e4:Event {event_id: "E004", name: "Visualize This: A Data Viz Workshop", description: "A workshop on advanced data visualization techniques.", date: "2024-04-05", location: "Innovation Hub, Lakeside"})
'''
session.run(query)

# Comment nodes
query = '''
CREATE (c1:Comment {comment_id: "C001", text: "Great post! Very informative.", date: "2023-11-02", seen_status: false})
CREATE (c2:Comment {comment_id: "C002", text: "Looking forward to this event.", date: "2023-11-06", seen_status: true})
CREATE (c3:Comment {comment_id: "C003", text: "Insightful article on big data!", date: "2023-11-11", seen_status: false})
CREATE (c4:Comment {comment_id: "C004", text: "Can't wait for the workshop on data visualization.", date: "2023-11-13", seen_status: true})
CREATE (c5:Comment {comment_id: "C005", text: "Really insightful post on ML algorithms!", date: "2023-11-16", seen_status: true})
CREATE (c6:Comment {comment_id: "C006", text: "The future of AI is both exciting and a bit daunting.", date: "2023-11-18", seen_status: false})
CREATE (c7:Comment {comment_id: "C007", text: "Data privacy is such a critical topic today. Thanks for this post!", date: "2023-11-21", seen_status: true})
CREATE (c8:Comment {comment_id: "C008", text: "Been looking for a good list of data viz tools. This is great!", date: "2023-11-23", seen_status: false})
CREATE (c9:Comment {comment_id: "C009", text: "Blockchain's role in data security cannot be overstated. Fascinating read.", date: "2023-11-26", seen_status: true})
CREATE (c10:Comment {comment_id: "C010", text: "Predictive analytics is evolving rapidly. Great to stay updated with posts like these.", date: "2023-11-29", seen_status: false})
'''
session.run(query)

### Edges

In [87]:
# FRIENDS_WITH (User ↔ User)
query1 = '''
MATCH (u1:User {user_id: "U001"}), (u2:User {user_id: "U002"})
CREATE (u1)-[:FRIENDS_WITH {since: "2021-06-15"}]->(u2)
CREATE (u2)-[:FRIENDS_WITH {since: "2021-06-15"}]->(u1)
'''

query2 = '''
MATCH (u3:User {user_id: "U003"}), (u4:User {user_id: "U004"})
CREATE (u3)-[:FRIENDS_WITH {since: "2020-08-10"}]->(u4)
CREATE (u4)-[:FRIENDS_WITH {since: "2020-08-10"}]->(u3)
'''
session.run(query2)

query3 = '''
MATCH (u1:User {user_id: "U001"}), (u3:User {user_id: "U003"})
CREATE (u1)-[:FRIENDS_WITH {since: "2022-02-05"}]->(u3)
CREATE (u3)-[:FRIENDS_WITH {since: "2022-02-05"}]->(u1)
'''
session.run(query3)

# query4 = '''
# MATCH (u1:User {user_id: "U001"}), (u4:User {user_id: "U004"})
# CREATE (u1)-[:FRIENDS_WITH {since: "2021-07-10"}]->(u4)
# CREATE (u4)-[:FRIENDS_WITH {since: "2021-07-10"}]->(u1)
# '''
# session.run(query4)

query5 = '''
MATCH (u2:User {user_id: "U002"}), (u5:User {user_id: "U005"})
CREATE (u2)-[:FRIENDS_WITH {since: "2020-11-22"}]->(u5)
CREATE (u5)-[:FRIENDS_WITH {since: "2020-11-22"}]->(u2)
'''
session.run(query5)

In [88]:
# CREATED (User → Post)
query = '''
WITH [
    {userId: "U001", postId: "P001"},
    {userId: "U002", postId: "P002"},
    {userId: "U003", postId: "P003"},
    {userId: "U003", postId: "P004"},
    {userId: "U004", postId: "P005"},
    {userId: "U005", postId: "P006"},
    {userId: "U003", postId: "P007"},
    {userId: "U001", postId: "P008"},
    {userId: "U002", postId: "P009"},
    {userId: "U003", postId: "P010"}
] AS userPostPairs
UNWIND userPostPairs AS userPostPair
MATCH (u:User {user_id: userPostPair.userId}), (p:Post {post_id: userPostPair.postId})
CREATE (u)-[:CREATED]->(p)
'''
session.run(query)

In [89]:
# ATTENDING (User → Event)
query = '''
WITH [
    {userId: "U001", eventId: "E001", rsvpDate: "2023-12-01"},
    {userId: "U002", eventId: "E002", rsvpDate: "2023-12-05"},
    {userId: "U001", eventId: "E002", rsvpDate: "2023-12-10"},
    {userId: "U002", eventId: "E003", rsvpDate: "2023-12-15"},
    {userId: "U003", eventId: "E004", rsvpDate: "2023-12-20"},
    {userId: "U004", eventId: "E001", rsvpDate: "2023-11-30"},
    {userId: "U005", eventId: "E002", rsvpDate: "2023-12-05"},
    {userId: "U006", eventId: "E003", rsvpDate: "2024-01-05"}
] AS userEventPairs
UNWIND userEventPairs AS userEventPair
MATCH (u:User {user_id: userEventPair.userId}), (e:Event {event_id: userEventPair.eventId})
CREATE (u)-[:ATTENDING {rsvp_date: userEventPair.rsvpDate}]->(e)
'''
session.run(query)

In [90]:
# COMMENTED_ON (User → Comment → Post)
query = '''
WITH [
    {userId: "U001", commentId: "C001", postId: "P001", date: "2023-11-02"},
    {userId: "U002", commentId: "C002", postId: "P001", date: "2023-11-06"},
    {userId: "U003", commentId: "C003", postId: "P002", date: "2023-11-11"},
    {userId: "U004", commentId: "C004", postId: "P003", date: "2023-11-13"},
    {userId: "U005", commentId: "C005", postId: "P004", date: "2023-11-16"},
    {userId: "U006", commentId: "C006", postId: "P005", date: "2023-11-18"},
    {userId: "U001", commentId: "C007", postId: "P006", date: "2023-11-21"},
    {userId: "U002", commentId: "C008", postId: "P007", date: "2023-11-23"},
    {userId: "U003", commentId: "C009", postId: "P008", date: "2023-11-26"},
    {userId: "U004", commentId: "C010", postId: "P009", date: "2023-11-29"}
] AS userCommentPostTriples
UNWIND userCommentPostTriples AS ucp
MATCH (u:User {user_id: ucp.userId}), (c:Comment {comment_id: ucp.commentId}), (p:Post {post_id: ucp.postId})
CREATE (u)-[:COMMENTED_ON {date: ucp.date}]->(c)-[:COMMENTED_ON {date: ucp.date}]->(p)
'''
session.run(query)


In [91]:
# FOLLOWING (User → User)
query = '''
WITH [
    {followerId: "U001", followedId: "U003", since: "2022-01-01"},
    {followerId: "U002", followedId: "U004", since: "2022-05-20"},
    {followerId: "U001", followedId: "U005", since: "2023-05-01"},
    {followerId: "U002", followedId: "U001", since: "2023-03-15"},
    {followerId: "U003", followedId: "U001", since: "2022-12-10"},
    {followerId: "U004", followedId: "U002", since: "2023-02-20"},
    {followerId: "U005", followedId: "U003", since: "2023-04-05"},
    {followerId: "U003", followedId: "U004", since: "2023-01-30"}
] AS userFollowPairs
UNWIND userFollowPairs AS ufp
MATCH (follower:User {user_id: ufp.followerId}), (followed:User {user_id: ufp.followedId})
CREATE (follower)-[:FOLLOWING {since: ufp.since}]->(followed)
'''
session.run(query)

In [92]:
# HOSTS (User → Event)
query = '''
WITH [
    {userId: "U004", eventId: "E003", date: "2023-11-10"},
    {userId: "U005", eventId: "E004", date: "2023-11-15"},
    {userId: "U002", eventId: "E001", date: "2023-11-30"},
    {userId: "U003", eventId: "E002", date: "2023-12-10"},
    {userId: "U001", eventId: "E003", date: "2024-01-05"}
] AS userEventPairs
UNWIND userEventPairs AS uep
MATCH (u:User {user_id: uep.userId}), (e:Event {event_id: uep.eventId})
CREATE (u)-[:HOSTS {date: uep.date}]->(e)
'''
session.run(query)

## Data Retrieval & Querying

### Retrieve a property of a specific User

In [93]:
query = '''
MATCH (u:User {user_id: "U001"})
RETURN u.email
'''
df = run_query(query)
df

,u.email
0,alicej@example.com


### Find all Posts created by a specific User

In [94]:
query = '''
MATCH (u:User {name: "Carol Lee"})-[:CREATED]->(post:Post)
RETURN post.title, post.body, post.date, post.type
'''
df = run_query(query)
df

,post.title,post.body,post.date,post.type
0,Big Data and Its Impact,Discussing the impact of Big Data on various i...,2023-11-10,video
1,Advancements in Predictive Analytics,Discussing recent advancements and trends in p...,2023-11-28,link
2,Data Privacy in the Digital Age,A comprehensive look at data privacy concerns ...,2023-11-20,text
3,Visualizing Data Effectively,Tips on creating effective data visualizations...,2023-11-12,image


### Find all Users who posted a specific topic of Post (topic is a property of Post)

In [95]:
query = '''
MATCH (u:User)-[:CREATED]->(p:Post)
WHERE p.title CONTAINS "Data" OR p.body CONTAINS "Data"
RETURN u.name, p.title, p.body, p.date, p.type
'''
df = run_query(query)
df

,u.name,p.title,p.body,p.date,p.type
0,Alice Johnson,Introduction to Data Science,Exploring the basics of Data Science...,2023-11-01,text
1,Bob Smith,Latest Trends in Data Analytics,An overview of recent developments in Data Ana...,2023-11-05,link
2,Carol Lee,Big Data and Its Impact,Discussing the impact of Big Data on various i...,2023-11-10,video
3,Carol Lee,Visualizing Data Effectively,Tips on creating effective data visualizations...,2023-11-12,image
4,Carol Lee,Data Privacy in the Digital Age,A comprehensive look at data privacy concerns ...,2023-11-20,text
5,Alice Johnson,Top 10 Data Visualization Tools,Ranking and reviewing the top 10 tools for dat...,2023-11-22,link
6,Bob Smith,Blockchain and Data Security,How blockchain technology is revolutionizing d...,2023-11-25,text


### Find common interests between two specific Users.

In [96]:
query = '''
MATCH (u1:User {user_id: "U001"}), (u2:User {user_id: "U005"})
WHERE u1.interest = u2.interest
RETURN u1.interest
'''
df = run_query(query)
df

,u1.interest
0,Data Science


### Retrieve top 3 Users who created most Posts

In [97]:
query = '''
MATCH (u:User)-[:CREATED]->(p:Post)
RETURN u.name, COUNT(p) AS posts_count
ORDER BY posts_count DESC
LIMIT 3
'''
df = run_query(query)
df

,u.name,posts_count
0,Carol Lee,4
1,Bob Smith,2
2,Alice Johnson,2


### Retrieve Users who haven’t created any Posts

In [98]:
query = '''
MATCH (u:User)
WHERE NOT EXISTS ((u)-[:CREATED]->(:Post))
RETURN u.name AS name
'''
df = run_query(query)
df

,name
0,Frank Harris


### Given two Users, identify if they are indirectly connected through a chain of friends and, if so, return the connecting path. For instance, can you find a relationship path where Alice is friends with someone (say, Bob), who in turn is friends with Charlie?

In [ ]:
# FRIENDS_WITH (User ↔ User)
query003_004 = '''
MATCH (u3:User {user_id: "U003"}), (u4:User {user_id: "U004"})
CREATE (u3)-[:FRIENDS_WITH {since: "2020-08-10"}]->(u4)
CREATE (u4)-[:FRIENDS_WITH {since: "2020-08-10"}]->(u3)
'''

query001_003 = '''
MATCH (u1:User {user_id: "U001"}), (u3:User {user_id: "U003"})
CREATE (u1)-[:FRIENDS_WITH {since: "2022-02-05"}]->(u3)
CREATE (u3)-[:FRIENDS_WITH {since: "2022-02-05"}]->(u1)
'''

In [101]:
cypher_query = """
MATCH path = shortestPath((u1:User {user_id: "U001"})-[:FRIENDS_WITH*]->(u2:User {user_id: "U004"}))
WHERE length(path) > 1
RETURN [node in nodes(path) | node.user_id] AS user_ids
"""

def find_indirect_connection(session):
    result = session.run(cypher_query)
    connecting_paths = []
    for record in result:
        user_ids = record["user_ids"]
        connecting_paths.append(user_ids)
    return connecting_paths

with GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password")) as driver:
    with driver.session() as session:
        connecting_paths = find_indirect_connection(session)
        for user_ids in connecting_paths:
            print(" -> ".join(user_ids))

U001 -> U003 -> U004


### Write a query to identify orphaned Users (Users who have no connections: in this case, no friends and now followers)

In [26]:
query = '''
MATCH (u:User)
WHERE NOT EXISTS ((u)-[:FRIENDS_WITH]-(:User)) AND NOT EXISTS ((u)-[:FOLLOWING]-(:User))
RETURN u.name AS name
'''
df = run_query(query)
df

,name
0,Frank Harris


### Can you think of a scenario when this information is useful?

Orphaned users are those who are not engaged with the platform’s network. This information is useful to identify new users who might need engagement or support, or to detect inactive or potentially fake accounts.?